<a href="https://colab.research.google.com/github/rahiakela/automl-experiments/blob/main/auto-keras-practice-works/01_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Image Classification

Reference:

https://autokeras.com/tutorial/image_classification/

In [1]:
!pip -q install autokeras

     |████████████████████████████████| 166 kB 7.7 MB/s 
     |████████████████████████████████| 454.4 MB 20 kB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 1.2 MB 55.7 MB/s 
     |████████████████████████████████| 462 kB 61.3 MB/s 
     |████████████████████████████████| 4.0 MB 46.2 MB/s 


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

##A Simple Example

The first step is to prepare your data. Here we use the MNIST dataset as an example

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])    # array([7, 2, 1], dtype=uint8)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


The second step is to run the ImageClassifier. It is recommended have more trials for more complicated datasets. This is just a quick demo of MNIST, so we set max_trials to 1. 

For the same reason, we set epochs to 10. You can also leave the epochs unspecified for an adaptive number of epochs.

In [4]:
# Initialize the image classifier
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
#  Feed the image classifier with training data
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 30m 18s]
val_loss: 0.039209458976984024

Best val_loss So Far: 0.039209458976984024
Total elapsed time: 00h 30m 18s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 180s 96ms/step - loss: 0.1564 - accuracy: 0.9524
Epoch 2/10
1875/1875 [==============================] - 180s 96ms/step - loss: 0.0736 - accuracy: 0.9774
Epoch 3/10
1875/1875 [==============================] - 175s 94ms/step - loss: 0.0591 - accuracy: 0.9819
Epoch 4/10
1875/1875 [==============================] - 177s 94ms/step - loss: 0.0514 - accuracy: 0.9839
Epoch 5/10
1875/1875 [==============================] - 181s 96ms/step - loss: 0.0464 - accuracy: 0.9850
Epoch 6/10
1875/1875 [==============================] - 180s 96ms/step - loss: 0.0406 - accuracy: 0.9866
Epoch 7/10
1875/1875 [==============================] - 180s 96ms/step - loss: 0.0393 - accuracy: 0.9876
Epoch 8/10
1875/1875 [==============================] - 174s 93ms/step - loss: 0.0354 - a

In [5]:
# Predict with the best model
predicted_y = clf.predict(x_test)
print(predicted_y)

313/313 [==============================] - 8s 26ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]


In [6]:
# Evaluate the best model with testing data
print(clf.evaluate(x_test, y_test))

313/313 [==============================] - 8s 24ms/step - loss: 0.0342 - accuracy: 0.9899
[0.03422395884990692, 0.9898999929428101]


##Validation Data

By default, AutoKeras use the last 20% of training data as validation data. 

As shown in the example below, you can use validation_split to specify the percentage.

In [7]:
# Split the training data and use the last 15% as validation data
clf.fit(x_train, y_train, validation_split=0.15, epochs=10)

You can also use your own validation set instead of splitting it from the training data with validation_data.

In [8]:
split = 50000

x_val = x_train[split:]
y_val = y_train[split:]

x_train = x_train[:split]
y_train = y_train[:split]

clf.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10)

##Customized Search Space

For advanced users, you may customize your search space by using AutoModel instead of ImageClassifier. You can configure the ImageBlock for some high-level configurations, e.g., block_type for the type of neural network to search, normalize for whether to do data normalization, augment for whether to do data augmentation. 

You can also do not specify these arguments, which would leave the different choices to be tuned automatically. 

See the following example for detail.

In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    block_type="resnet", # Only search ResNet architecture
    normalize=True,      # Normalize the dataset
    augment=False        # Do not do data augmentation
)(input_node)

output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)

clf.fit(x_train, y_train, epochs=10)

The usage of AutoModel is similar to the functional API of Keras. Basically, you are building a graph, whose edges are blocks and the nodes are intermediate outputs of blocks. To add an edge from `input_node` to `output_node` with `output_node = ak.some_block(input_node)`.

You can even also use more fine grained blocks to customize the search space even further. 

See the following example.

In [ ]:
input_node = ak.ImageInput()
output_node = ak.Normalization(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)
clf.fit(x_train, y_train, epochs=10)

##Data Format

The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel dimension. The images in the MNIST dataset do not have the channel dimension. Each image is a matrix with shape (28, 28). AutoKeras also accepts images of three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28, 28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and 1s.

So if you prepare your data in the following way, the ImageClassifier should still work.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension
x_train = x_train.reshape(x_train.shape + (1, ))
x_test = x_test.reshape(x_test.shape + (1, ))

# One-hot encode the labels
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print(y_train[:3])

In [ ]:
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the classifier.
clf.fit(x_train, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test))

We also support using tf.data.Dataset format for the training data.

In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train, ), (y_train, )))
test_set = tf.data.Dataset.from_tensor_slices(((x_test, ), (y_test, )))

clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))